In [13]:
import torch
import random
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.dynamic_module_utils import get_imports
from datasets import load_dataset
from unittest.mock import patch
import os


def fixed_get_imports(filename: str | os.PathLike) -> list[str]:
    imports = get_imports(filename)
    if not torch.cuda.is_available() and "flash_attn" in imports:
        imports.remove("flash_attn")
    return imports

class DeepSeekMMLUEvaluation:
    def __init__(self, model_name="deepseek-ai/DeepSeek-R1", dataset_name="tasksource/mmlu"):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model_name = 'qnguyen3/nanoLLaVA-1.5'
        with patch("transformers.dynamic_module_utils.get_imports", fixed_get_imports):
            self.model = AutoModelForCausalLM.from_pretrained(
                    model_name,
                    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                    device_map='auto' if torch.cuda.is_available() else None,
                    trust_remote_code=True
                )
            self.tokenizer = AutoTokenizer.from_pretrained(
                        model_name,
                        trust_remote_code=True
                    )
        dataset = load_dataset("tasksource/mmlu", "anatomy")
        self.dataset = dataset
    
    def preprocess(self, text):
        inputs = self.tokenizer(text, return_tensors="pt").to(self.device)
        return inputs
    
    def evaluate(self, split="validation"):
        dataset_split = self.dataset[split]
        correct = 0
        total = len(dataset_split)
        
        for item in dataset_split:
            question = item["question"]
            choices = item["choices"]
            correct_answer = item["answer"]
            
            inputs = self.preprocess(question + "\n" + "\n".join(choices))
            with torch.no_grad():
                outputs = self.model(**inputs)
            logits = outputs.logits[:, -1, :]
            predicted_answer = torch.argmax(logits).item()
            
            if predicted_answer == correct_answer:
                correct += 1
        
        accuracy = correct / total * 100
        return accuracy
    
    def inject_random_bit_flip(self):
        import numpy as np
        param_name, param_tensor = random.choice(list(self.model.named_parameters()))
        param_data = param_tensor.detach().cpu().numpy()
        param_flat = param_data.flatten()
        byte_index = random.randint(0, param_flat.nbytes - 1) 
        bit_position = random.randint(0, 7)  
        param_as_bytes = np.frombuffer(param_flat.view(np.uint8), dtype=np.uint8)
        param_as_bytes[byte_index] ^= 1 << bit_position
        modified_param = torch.tensor(
            param_flat.reshape(param_data.shape), dtype=param_tensor.dtype
        )
        param_tensor.data = modified_param.to(param_tensor.device)

        return param_name, byte_index, bit_position
    
    def run_experiment(self, num_flips=5):
        original_accuracy = self.evaluate()
        print(f"Original MMLU Accuracy: {original_accuracy:.2f}%")
        
        flip_results = []
        for i in range(num_flips):
            param_name, byte_index, bit_position = self.inject_random_bit_flip()
            new_accuracy = self.evaluate()
            flip_results.append((param_name, byte_index, bit_position, new_accuracy))
            print(f"Flip {i+1}: {param_name}, Byte {byte_index}, Bit {bit_position}, Accuracy: {new_accuracy:.2f}%")
        
        return flip_results


In [7]:
deepseek=DeepSeekMMLUEvaluation()
deepseek.run_experiment()

Original MMLU Accuracy: 0.00%
Flip 1: model.layers.7.self_attn.q_proj.weight, Byte 1610207, Bit 4, Accuracy: 0.00%
Flip 2: model.layers.19.mlp.down_proj.weight, Byte 9689436, Bit 4, Accuracy: 0.00%
Flip 3: model.layers.1.self_attn.o_proj.weight, Byte 3591366, Bit 6, Accuracy: 0.00%
Flip 4: model.layers.18.self_attn.k_proj.weight, Byte 3943560, Bit 5, Accuracy: 0.00%
Flip 5: model.layers.15.self_attn.v_proj.weight, Byte 3364119, Bit 5, Accuracy: 0.00%


[('model.layers.7.self_attn.q_proj.weight', 1610207, 4, 0.0),
 ('model.layers.19.mlp.down_proj.weight', 9689436, 4, 0.0),
 ('model.layers.1.self_attn.o_proj.weight', 3591366, 6, 0.0),
 ('model.layers.18.self_attn.k_proj.weight', 3943560, 5, 0.0),
 ('model.layers.15.self_attn.v_proj.weight', 3364119, 5, 0.0)]

In [3]:
dataset = load_dataset("tasksource/mmlu", "anatomy")

README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

mmlu.py:   0%|          | 0.00/5.20k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/19.7k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/3.08k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/135 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/14 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

In [18]:
import torch
import random
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader

class DeepSeekMMLUEvaluation:
    def __init__(self, model_name="deepseek-ai/DeepSeek-R1", device=None):
        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1", trust_remote_code=True)
        self.original_state_dict = {k: v.clone() for k, v in self.model.state_dict().items()}
    
    def format_mmlu_question(self, question, choices):
        """Formats the MMLU question for causal LM input."""
        formatted = f"Question: {question}\n"
        for i, choice in enumerate(choices):
            formatted += f"{chr(65+i)}) {choice}\n"
        formatted += "Answer: "
        return formatted
    
    def evaluate_choice_log_likelihood(self, question, choices):
        """Evaluates the log-likelihood for each multiple-choice answer."""
        prompt = self.format_mmlu_question(question, choices)
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        choice_probs = []
        
        for choice in choices:
            choice_input = self.tokenizer(choice, return_tensors="pt").to(self.device)
            with torch.no_grad():
                outputs = self.model(**inputs, labels=inputs.input_ids)
                log_probs = outputs.loss.item()
                choice_probs.append(-log_probs)  # Negate loss to get likelihood
        
        return choices[np.argmax(choice_probs)]  # Return most likely choice
    
    def evaluate(self, dataset):
        """Evaluates model accuracy on MMLU dataset."""
        correct = 0
        total = 0
        
        for question, choices, answer in dataset:
            predicted_answer = self.evaluate_choice_log_likelihood(question, choices)
            if predicted_answer == answer:
                correct += 1
            total += 1
        
        return (correct / total) * 100
    
    def inject_random_bit_flip(self):
        """Injects a random bit flip into the model weights."""
        param_name, param_tensor = random.choice(list(self.model.named_parameters()))
        param_data = param_tensor.detach().cpu().numpy()
        param_flat = param_data.flatten()
        byte_index = random.randint(0, param_flat.nbytes - 1)
        bit_position = random.randint(0, 7)
        
        param_as_bytes = np.frombuffer(param_flat.view(np.uint8), dtype=np.uint8)
        param_as_bytes[byte_index] ^= 1 << bit_position
        
        modified_param = torch.tensor(param_flat.reshape(param_data.shape), dtype=param_tensor.dtype).to(self.device)
        param_tensor.data = modified_param
        
        return param_name, byte_index, bit_position
    
    def reset_model_weights(self):
        """Resets model weights to their original state."""
        with torch.no_grad():
            for name, param in self.model.named_parameters():
                param.copy_(self.original_state_dict[name])
    
    def run_experiment(self, dataset, num_flips=10):
        """Runs the accuracy evaluation with and without bit flips."""
        original_accuracy = self.evaluate(dataset)
        print(f"Original Accuracy: {original_accuracy:.2f}%")
        
        flip_results = []
        for i in range(num_flips):
            param_name, byte_index, bit_position = self.inject_random_bit_flip()
            new_accuracy = self.evaluate(dataset)
            flip_results.append((param_name, byte_index, bit_position, new_accuracy))
            print(f"Flip {i+1}: {param_name}, Byte {byte_index}, Bit {bit_position}, Accuracy: {new_accuracy:.2f}%")
            self.reset_model_weights()
        
        return flip_results


In [19]:
deepseek_eval = DeepSeekMMLUEvaluation()

# Load MMLU dataset
train_dataset = deepseek_eval.load_mmlu(subset="high_school_math", split="train")
val_dataset = deepseek_eval.load_mmlu(subset="high_school_math", split="test")

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8)

# Train the model
deepseek_eval.train(train_dataloader)

# Run bit-flip experiment
deepseek_eval.run_experiment(val_dataloader, num_flips=5)


ValueError: Unknown quantization type, got fp8 - supported types are: ['awq', 'bitsandbytes_4bit', 'bitsandbytes_8bit', 'gptq', 'aqlm', 'quanto', 'eetq', 'higgs', 'hqq', 'compressed-tensors', 'fbgemm_fp8', 'torchao', 'bitnet', 'vptq']